In [3]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.2/265.2 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.44.0.3-py2.py3-none-any.whl size=265293968 sha256=faac25fefced57889337685bf214d903f3b159a7487ae4683b58b99cca0ffa86
  Stored in directory: /root/.cache/pip/wheels/77/9a/1c/2da26f943fd46b57f3c20b54847b936b9152b831dc7447cf71
Successfully built h2o


In [ ]:
# H2O AutoMl

import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Start H2O
h2o.init()

# dataset
df = pd.read_csv('final2.csv')

target_variable = 'DiagPeriodL90D'

train, test = train_test_split(df, test_size=0.2, random_state=42)

# Pandas DataFrame to an H2O Frame
train_h2o = h2o.H2OFrame(train)
test_h2o = h2o.H2OFrame(test)

# Defining predictors
predictors = list(df.columns[df.columns != target_variable])

#AutoML model
aml = H2OAutoML(max_runtime_secs=600)  # Adjust max_runtime_secs as needed

# Train
aml.train(x=predictors, y=target_variable, training_frame=train_h2o)

#leaderboard
lb = aml.leaderboard
print(lb)

# prediction
predictions_h2o = aml.leader.predict(test_h2o)

# H2O predictions to a Pandas DataFrame
predictions_df = h2o.as_list(predictions_h2o)

# Ensuring predictions are in the correct format (0 or 1)
predictions_df['predict'] = predictions_df['predict'].astype(int)

# Evaluation
accuracy = accuracy_score(test[target_variable], predictions_df['predict'])
print(f'Accuracy: {accuracy}')

# classification report
print(classification_report(test[target_variable], predictions_df['predict']))

# Shut down H2O
h2o.shutdown()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp7pbenei6
  JVM stdout: /tmp/tmp7pbenei6/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp7pbenei6/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 21 days
H2O_cluster_name:,H2O_from_python_unknownUser_uv8t48
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
07:31:59.118: _train param, Dropping bad and constant columns: [patient_F]
07:31:59.118: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

█████
07:32:40.531: _train param, Dropping bad and constant columns: [patient_F]
07:32:40.531: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

█
07:32:48.526: _train param, Dropping bad and constant columns: [patient_F]
07:32:48.526: _response param, We have detected that your response column has only 2 unique 

In [1]:
# Keras

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow import keras
from tensorflow.keras import layers

df = pd.read_csv('final2.csv')

target_variable = 'DiagPeriodL90D'

train, test = train_test_split(df, test_size=0.2, random_state=42)

#  predictors
predictors = list(df.columns[df.columns != target_variable])

X_train = train[predictors].values
y_train = train[target_variable].values
X_test = test[predictors].values
y_test = test[target_variable].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Keras model
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# predictions
predictions = model.predict(X_test)
predictions_binary = (predictions > 0.5).astype(int)

# Evaluation
accuracy = accuracy_score(y_test, predictions_binary)
print(f'Accuracy: {accuracy}')

# classification report
print(classification_report(y_test, predictions_binary))


Epoch 1/50
259/259 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.5921 - val_loss: nan - val_accuracy: 0.3787
Epoch 2/50
259/259 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.3752 - val_loss: nan - val_accuracy: 0.3787
Epoch 3/50
259/259 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.3752 - val_loss: nan - val_accuracy: 0.3787
Epoch 4/50
259/259 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.3752 - val_loss: nan - val_accuracy: 0.3787
Epoch 5/50
259/259 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.3752 - val_loss: nan - val_accuracy: 0.3787
Epoch 6/50
259/259 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.3752 - val_loss: nan - val_accuracy: 0.3787
Epoch 7/50
259/259 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.3752 - val_loss: nan - val_accuracy: 0.3787
Epoch 8/50
259/259 [============================

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
